# Code Generation and Evaluation

In [1]:
from human_eval.data import write_jsonl, read_problems
from human_eval.evaluation import evaluate_functional_correctness
import os
import logging
import sys
import random, time, datetime
from mistralai import Mistral
import json

%load_ext autoreload
%autoreload 2

In [2]:
from prompts import ( complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,
                      complete_task_prompt_basic, complete_task_prompt, complete_task_prompt_full, )
#from helpers import clean_code, write_jsonl

In [3]:
# select model, propmt, temperature, top_p
models = [
    ('open-mistral-7b', 'mistral_7b'),
    ('ministral-3b-latest', 'mistral_3b'),
    ('ministral-8b-latest', 'mistral_8B'),
    ('open-mistral-nemo', 'nemo'),
    ('open-codestral-mamba', 'codestral_mamba'),
    ]
# actual prompt & prompt name for log file name
prompts_and_names = [
    (complete_code_prompt_basic, 'complete_code_prompt_basic'),            
    (complete_code_prompt,       'complete_code_prompt'),
    (complete_code_prompt_full,  'complete_code_prompt_full'),
    (complete_task_prompt_basic, 'complete_task_prompt_basic'),
    (complete_task_prompt,       'complete_task_prompt'),
    (complete_task_prompt_full,  'complete_task_prompt_full'), ]
# temperature and top_k values and labels for file names
temperature_values = [
    (1.0,  'temperature1.0'),
    (0.75, 'temperature0.75'),
    (0.5,  'temperature0.5'), ]
top_p_values = [
    (1.0,  'topP1.0'),
    (0.75, 'topP0.75'),
    (0.5,  'topP0.5'), ]

model_idx       = 4
prompt_idx      = 3
temperature_idx = 0
top_p_idx       = 0

model_name, model_nickname     = models[ model_idx ]
my_prompt, my_prompt_label     = prompts_and_names[ prompt_idx ]
TEMPERATURE, temperature_label = temperature_values[ temperature_idx ]
TOP_P, top_p_label             = top_p_values[ top_p_idx ]

SPECIAL_MESSAGE = f'Model temperature: {TEMPERATURE}. Model top_p: {TOP_P}. Model top_k: 50. Model dtype: torch.float32'

print(f'Model name:     {model_name}')
print(f'Model nickname: {model_nickname}')
print(f'Prompt name:    {my_prompt_label}')
print(f'Prompt:\n{my_prompt.strip().format("Task Description", "Test Cases")}\n')
print(f'Temperature: {TEMPERATURE}. Temperature label: {temperature_label}')
print(f'Top p: {TOP_P}. Top p label: {top_p_label}')
print(f'Special message: {SPECIAL_MESSAGE}')

Model name:     open-codestral-mamba
Model nickname: codestral_mamba
Prompt name:    complete_task_prompt_basic
Prompt:
Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

Temperature: 1.0. Temperature label: temperature1.0
Top p: 1.0. Top p label: topP1.0
Special message: Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32


In [4]:
# save results
time_stamp     = datetime.datetime.now().strftime("%Y%m%d_%H%M%S_%f")[:-2]
results_file   = f'logs_local/MBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_completions_{time_stamp}.jsonl'
log_file       = f'logs_local/logs/MBPP_{model_nickname}_{my_prompt_label}_{temperature_label}_{top_p_label}_log_{time_stamp}.log'

# log results
for handler in logging.root.handlers[:]:            # overwrite any previous handlers with different formats
    logging.root.removeHandler(handler)
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s  %(levelname)s  %(message)s',
    handlers=[
        logging.FileHandler(log_file),              # Log to a file
        logging.StreamHandler(sys.stdout)           # Log to console (default - sys.stderr (red background)
    ]
)

# print job config
logging.info('*****     JOB CONFIGURATION     *****')
logging.info(f'MODEL: {model_name}')
logging.info(f'MODEL NICKNAME: {model_nickname}')
logging.info(SPECIAL_MESSAGE)
logging.info(f'GENERATED CODE SAVED IN: {results_file}')
logging.info(f'THIS LOG FILE: {log_file}')
if my_prompt in [complete_code_prompt_basic, complete_code_prompt, complete_code_prompt_full,]:
    logging.info(f'PROMPT:\n{my_prompt.lstrip().format("Starter Code")}')
else:
    logging.info(f'\nPROMPT:\n{my_prompt.format("Task Description", "Test Cases")}')
logging.info('*****     END OF JOB CONFIGURATION     *****')

2025-04-05 03:09:38,438  INFO  *****     JOB CONFIGURATION     *****
2025-04-05 03:09:38,438  INFO  MODEL: open-codestral-mamba
2025-04-05 03:09:38,439  INFO  MODEL NICKNAME: codestral_mamba
2025-04-05 03:09:38,439  INFO  Model temperature: 1.0. Model top_p: 1.0. Model top_k: 50. Model dtype: torch.float32
2025-04-05 03:09:38,439  INFO  GENERATED CODE SAVED IN: logs_local/MBPP_codestral_mamba_complete_task_prompt_basic_temperature1.0_topP1.0_completions_20250405_030938_4378.jsonl
2025-04-05 03:09:38,440  INFO  THIS LOG FILE: logs_local/logs/MBPP_codestral_mamba_complete_task_prompt_basic_temperature1.0_topP1.0_log_20250405_030938_4378.log
2025-04-05 03:09:38,440  INFO  
PROMPT:

Complete the following task:
Task Description.

Your output code must satisfy these tests:
Test Cases

2025-04-05 03:09:38,440  INFO  *****     END OF JOB CONFIGURATION     *****


In [5]:
# read dataset
file = 'data/MBPP_Test.jsonl.gz'
tasks = read_problems(file)
print(f'Type of tasks: {type(tasks)}\n')

counter = 0
for k,v in tasks.items():
    if counter == 2:
        break
    print(f'task_id: {k}', type(k))
    for k2, v2 in v.items():
        print(f'\n{k2}:\n{v2}')
    print('\n' + '='*100 + '\n')
    counter += 1

Type of tasks: <class 'dict'>

task_id: 11 <class 'str'>

task_id:
11

test:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"

prompt:
Write a python function to remove first and last occurrence of a given character from the string.

canonical_solution:
def remove_Occ(s,ch): 
    for i in range(len(s)): 
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    for i in range(len(s) - 1,-1,-1):  
        if (s[i] == ch): 
            s = s[0 : i] + s[i + 1:] 
            break
    return s 

entry_point:
None


task_id: 12 <class 'str'>

task_id:
12

test:

assert sort_matrix([[1, 2, 3], [2, 4, 5], [1, 1, 1]])==[[1, 1, 1], [1, 2, 3], [2, 4, 5]]
assert sort_matrix([[1, 2, 3], [-2, 4, -5], [1, -1, 1]])==[[-2, 4, -5], [1, -1, 1], [1, 2, 3]]
assert sort_matrix([[5,8,9],[6,4,3],[2,1,4]])==[[2, 1, 4], [6, 4, 3], [5, 8, 9]]

prompt:
Write a function to sort a given matrix in ascending order accordi

In [7]:
def api_call(model, prompt):
    ''' One API call to a Mistral model '''
    chat_response = client.chat.complete(
                model=model,
                messages=[{
                        "role": "user",
                        "content": prompt, 
                    }]
                )
    return chat_response.choices[0].message.content

def generate_response_with_backoff(model, prompt, max_retries=10, initial_delay=1, max_delay=16):
    '''
    Make API calls with exponential backoff on errors.
    Args:
        model (str): The model identifier.
        prompt (str): The prompt to send.
        max_retries (int): Maximum number of retries before giving up.
        initial_delay (int or float): Starting delay in seconds.
        max_delay (int or float): Maximum delay between retries.
    Returns:
        str: The chat response content OR f'Error generating a completion:{e}'
        if the max number of attemps has been reached
    '''
    retries = 0
    delay   = initial_delay
    while True:
        try:
            return api_call(model, prompt)
        except Exception as e:
            retries += 1
            if retries > max_retries:
                return f"Error generating a completion after {retries} retries:\n{e}"
            sleep_time = delay + random.uniform(0, 1)    # add jitter
            time.sleep(sleep_time)
            delay = min(delay * 2, max_delay)

In [8]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [9]:
input_prompt = "What is the capital of California?"
print(input_prompt)
print(generate_response_with_backoff(model_name, input_prompt))

What is the capital of California?
2025-04-05 03:09:39,507  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
The capital of California is Sacramento. It is the capital of the U.S. state of California, located in the northern part of the state. The California State Capitol building is located in Sacramento, which serves as the state's administrative center.


In [10]:
num_samples_per_task = 1
completions  = []

for task_id, task_body in tasks.items():
    logging.info( task_id )
    #if task_id == '15':
    #        break
    for i in range(num_samples_per_task):
        start_time  = time.time()
        full_prompt = my_prompt.lstrip().format( task_body['prompt'], task_body['test'] )
        completion = generate_response_with_backoff( model_name, full_prompt )
        completions.append( {'task_id': task_id, 'completion': completion} )
        write_jsonl(results_file, completions)
                
        logging.info('PROMPT:\n' + full_prompt + '\n')
        logging.info('COMPLETION:\n' + completion + '\n')
        logging.info(f"Time elapsed including backoff: {(time.time() - start_time):.4f} seconds\n")

2025-04-05 03:09:39,556  INFO  11
2025-04-05 03:09:42,427  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:09:42,442  INFO  PROMPT:
Complete the following task:
Write a python function to remove first and last occurrence of a given character from the string..

Your output code must satisfy these tests:

assert remove_Occ("hello","l") == "heo"
assert remove_Occ("abcda","a") == "bcd"
assert remove_Occ("PHP","P") == "H"


2025-04-05 03:09:42,443  INFO  COMPLETION:
Sure, the following python function fulfills the given specifications:

```python
def remove_Occ(s, c):
    return s[:s.index(c)] + s[s.index(c) + 1:-s[::-1].index(c) - 1] + s[-s[::-1].index(c):]
```

This function works as follows:

1. It uses Python's built-in `index` method to find the index of the first occurrence of the character `c` in the string `s`.
2. It then uses `[::-1]` to reverse the string `s`, and again `index` to find the index of the last occurrence of `c` in t

2025-04-05 03:09:56,644  INFO  Time elapsed including backoff: 3.7566 seconds

2025-04-05 03:09:56,644  INFO  16
2025-04-05 03:10:00,040  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:10:00,050  INFO  PROMPT:
Complete the following task:
Write a function to find sequences of lowercase letters joined with an underscore..

Your output code must satisfy these tests:

assert text_lowercase_underscore("aab_cbbbc")==('Found a match!')
assert text_lowercase_underscore("aab_Abbbc")==('Not matched!')
assert text_lowercase_underscore("Aaab_abbbc")==('Not matched!')


2025-04-05 03:10:00,051  INFO  COMPLETION:
Here is a Python solution using regular expressions:

```python
import re

def text_lowercase_underscore(text):
    match = re.fullmatch(r'[a-z]+_[a-z]+', text)
    if match:
        return 'Found a match!'
    else:
        return 'Not matched!'
```

This solution works by using a regular expression to check if the text matches the patt

2025-04-05 03:10:12,547  INFO  Time elapsed including backoff: 2.5112 seconds

2025-04-05 03:10:12,549  INFO  22
2025-04-05 03:10:15,398  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:10:15,412  INFO  PROMPT:
Complete the following task:
Write a function to find the first duplicate element in a given array of integers..

Your output code must satisfy these tests:

assert find_first_duplicate(([1, 2, 3, 4, 4, 5]))==4
assert find_first_duplicate([1, 2, 3, 4])==-1
assert find_first_duplicate([1, 1, 2, 3, 3, 2, 2])==1


2025-04-05 03:10:15,414  INFO  COMPLETION:
Here is a Python function that accomplishes this task. It iterates through the array checking if each element is in a set. If it's already in the set, it's a duplicate and the function returns it. If no duplicates are found, it returns -1.

```python
def find_first_duplicate(array):
    seen = set()
    for num in array:
        if num in seen:
            return num
        see

2025-04-05 03:10:29,753  INFO  Time elapsed including backoff: 2.7623 seconds

2025-04-05 03:10:29,755  INFO  28
2025-04-05 03:10:34,142  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:10:34,154  INFO  PROMPT:
Complete the following task:
Write a python function to find binomial co-efficient..

Your output code must satisfy these tests:

assert binomial_Coeff(5,2) == 10
assert binomial_Coeff(4,3) == 4
assert binomial_Coeff(3,2) == 3


2025-04-05 03:10:34,155  INFO  COMPLETION:
Here is a Python function to calculate the binomial coefficient:

```python
def binomial_Coeff(n, k):
    res = 1

    # Since C(n, k) = C(n, n-k)
    if (k > n - k):
        k = n - k

    for i in range(k):
        res = res * (n - i)
        res = res // (i + 1)

    return res
```

You can test the function with the following code:

```python
assert binomial_Coeff(5,2) == 10
assert binomial_Coeff(4,3) == 4
assert binomial_Coeff(3,2) == 3
```

This function 

2025-04-05 03:10:50,332  INFO  COMPLETION:
Here is a Python function that converts decimal numbers to binary:

```python
def decimal_To_Binary(n):
    return bin(n).replace("0b", "")
```

This function uses the built-in `bin()` function in Python to convert a decimal number to binary. The `bin()` function returns a string that starts with "0b" to indicate that it is a binary number, so we use the `replace()` method to remove this prefix.

You can test this function with the following code:

```python
assert decimal_To_Binary(10) == '1010'
assert decimal_To_Binary(1) == '1'
assert decimal_To_Binary(20) == '10100'
```

2025-04-05 03:10:50,333  INFO  Time elapsed including backoff: 2.0225 seconds

2025-04-05 03:10:50,333  INFO  34
2025-04-05 03:10:53,290  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:10:53,302  INFO  PROMPT:
Complete the following task:
Write a python function to find the missing number in a sorted array..

Your output

...

2025-04-05 03:37:21,618  INFO  COMPLETION:
Here is the Python function that sorts a list of tuples in increasing order by the last element in each tuple:

```python
def sort_tuple(l):
    return sorted(l, key=lambda x: x[-1])
```

This function uses the built-in `sorted` function in Python. The `key` argument of the `sorted` function is a function that takes an element as input and returns a value that will be used for sorting. In this case, we use a `lambda` function that returns the last element of a tuple.

Let's test this function with the given test cases:

```python
print(sort_tuple([(1, 3), (3, 2), (2, 1)] ))
print(sort_tuple([(2, 4), (3, 3), (1, 1)] ))
print(sort_tuple([(3, 9), (6, 7), (4, 3)] ))
```

This will output:

```
[(2, 1), (3, 2), (1, 3)]
[(1, 1), (3, 3), (2, 4)]
[(4, 3), (6, 7), (3, 9)]
```

As we can see, the function correctly sorts the list of tuples in increasing order by the last element in each tuple.

2025-04-05 03:37:21,618  INFO  Time elapsed including backo

2025-04-05 03:37:36,877  INFO  493
2025-04-05 03:37:44,147  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:37:44,162  INFO  PROMPT:
Complete the following task:
Write a function to calculate a grid of hexagon coordinates where function returns a list of lists containing 6 tuples of x, y point coordinates..

Your output code must satisfy these tests:

assert calculate_polygons(1,1, 4, 4, 3)==[[(-5.0, -4.196152422706632), (-5.0, -0.7320508075688767), (-2.0, 1.0), (1.0, -0.7320508075688767), (1.0, -4.196152422706632), (-2.0, -5.928203230275509), (-5.0, -4.196152422706632)], [(1.0, -4.196152422706632), (1.0, -0.7320508075688767), (4.0, 1.0), (7.0, -0.7320508075688767), (7.0, -4.196152422706632), (4.0, -5.928203230275509), (1.0, -4.196152422706632)], [(7.0, -4.196152422706632), (7.0, -0.7320508075688767), (10.0, 1.0), (13.0, -0.7320508075688767), (13.0, -4.196152422706632), (10.0, -5.928203230275509), (7.0, -4.196152422706632)], [(-2.0, 1

2025-04-05 03:37:47,302  INFO  Time elapsed including backoff: 3.1365 seconds

2025-04-05 03:37:47,302  INFO  495
2025-04-05 03:37:49,961  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:37:49,978  INFO  PROMPT:
Complete the following task:
Write a function to remove lowercase substrings from a given string by using regex..

Your output code must satisfy these tests:

assert remove_lowercase('KDeoALOklOOHserfLoAJSIskdsf') == 'KDALOOOHLAJSI'
assert remove_lowercase('ProducTnamEstreAmIngMediAplAYer') == 'PTEAIMAAY'
assert remove_lowercase('maNufacTuredbYSheZenTechNolOGIes') == 'NTYSZTNOGI'


2025-04-05 03:37:49,979  INFO  COMPLETION:
Here is a Python solution using the `re` module:

```python
import re

def remove_lowercase(s):
    return re.sub(r'[a-z]', '', s)
```

This function uses a regular expression to match any lowercase letter (`[a-z]`), and replaces them with an empty string (effectively removing them from the string). The fun

2025-04-05 03:38:03,399  INFO  Time elapsed including backoff: 2.6467 seconds

2025-04-05 03:38:03,401  INFO  501
2025-04-05 03:38:06,755  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:38:06,777  INFO  PROMPT:
Complete the following task:
Write a python function to find common divisor between two numbers in a given pair..

Your output code must satisfy these tests:

assert num_comm_div(2,4) == 2
assert num_comm_div(2,8) == 2
assert num_comm_div(12,24) == 6


2025-04-05 03:38:06,778  INFO  COMPLETION:
Here is a Python function that finds the common divisor between two numbers in a given pair. This function uses the Euclidean algorithm to find the greatest common divisor (GCD) of the two numbers, which is the largest common divisor.

```python
def num_comm_div(a, b):
    # Function to calculate gcd
    def gcd(x, y):
        while(y):
            x, y = y, x % y
        return x

    return gcd(a, b)
```

The `gcd()` function uses the

2025-04-05 03:38:22,445  INFO  Time elapsed including backoff: 3.4804 seconds

2025-04-05 03:38:22,446  INFO  507
2025-04-05 03:38:25,187  INFO  HTTP Request: POST https://api.mistral.ai/v1/chat/completions "HTTP/1.1 200 OK"
2025-04-05 03:38:25,207  INFO  PROMPT:
Complete the following task:
Write a function to remove specific words from a given list..

Your output code must satisfy these tests:

assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['white', 'orange'])==['red', 'green', 'blue', 'black']
assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['black', 'orange'])==['red', 'green', 'blue', 'white']
assert remove_words(['red', 'green', 'blue', 'white', 'black', 'orange'],['blue', 'white'])==['red', 'green', 'black', 'orange']


2025-04-05 03:38:25,208  INFO  COMPLETION:
Here is a Python solution using a list comprehension.

```python
def remove_words(lst, words_to_remove):
    return [i for i in lst if i not in words_to_remove]
```

To

In [11]:
unknown_variable

NameError: name 'unknown_variable' is not defined

# Appendix

In [ ]:
# read a jsonl file
file_name = 'logs/mistral_7b_samples_20241201_005244_0462.jsonl'
data = read_jsonl(file_name)

In [ ]:
data[0]['completion']

In [ ]:
# modify and write back
data2 = []
for i in data:
    new_i = dict()
    new_i['task_id'] = i['task_id']
    new_i['completion'] = add_import(i['completion'])
    data2.append(new_i)

In [ ]:
file_name = 'logs/mistral_7b_samples_20241201_005244_0462_v2.jsonl'
write_jsonl(file_name, data2)